# Binned statistics
# Eddy advection and SSTA - 300km filter

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import cmocean as cmo
import glob
from tqdm import tqdm
import sys
import calendar

# Change case_name to recreate the other case

In [2]:
case_name = 'Weak'
years = '[2-3]'

In [3]:
x_array = np.arange(0, 1122*2500, 2500)
y_array = np.arange(20*2500, 430*2500, 2500)
x_rho, y_rho = np.meshgrid(x_array, y_array)

In [4]:
heat_balance_files = glob.glob('../monthly_ml_heat_budget_data_small_mesoscale/' 
                               + case_name + '/monthly_ml_heat_balance_2016-*_' + case_name + 'Eddies.nc')
heat_balance_files = sorted(heat_balance_files)
heat_balance_files

['../monthly_ml_heat_budget_data_small_mesoscale/Weak/monthly_ml_heat_balance_2016-01_WeakEddies.nc',
 '../monthly_ml_heat_budget_data_small_mesoscale/Weak/monthly_ml_heat_balance_2016-02_WeakEddies.nc',
 '../monthly_ml_heat_budget_data_small_mesoscale/Weak/monthly_ml_heat_balance_2016-03_WeakEddies.nc',
 '../monthly_ml_heat_budget_data_small_mesoscale/Weak/monthly_ml_heat_balance_2016-04_WeakEddies.nc',
 '../monthly_ml_heat_budget_data_small_mesoscale/Weak/monthly_ml_heat_balance_2016-05_WeakEddies.nc',
 '../monthly_ml_heat_budget_data_small_mesoscale/Weak/monthly_ml_heat_balance_2016-06_WeakEddies.nc',
 '../monthly_ml_heat_budget_data_small_mesoscale/Weak/monthly_ml_heat_balance_2016-07_WeakEddies.nc',
 '../monthly_ml_heat_budget_data_small_mesoscale/Weak/monthly_ml_heat_balance_2016-08_WeakEddies.nc',
 '../monthly_ml_heat_budget_data_small_mesoscale/Weak/monthly_ml_heat_balance_2016-09_WeakEddies.nc',
 '../monthly_ml_heat_budget_data_small_mesoscale/Weak/monthly_ml_heat_balance_2016

In [5]:
%%time
SSTA, eddy_adv_ml = np.zeros([len(heat_balance_files), 410, 1122]), np.zeros([len(heat_balance_files), 410, 1122])
for num, temp_rate_ml_eddy_file in enumerate(heat_balance_files):
    #print(temp_rate_ml_eddy_file)
    temp_rate_ml_ds = xr.open_dataset(temp_rate_ml_eddy_file)
    SSTA[num] = temp_rate_ml_ds.SSTA.data
    eddy_adv_ml[num] = temp_rate_ml_ds.eddy_adv_ml.data

CPU times: user 127 ms, sys: 64 ms, total: 191 ms
Wall time: 629 ms


In [6]:
MS_ds = xr.Dataset(data_vars={'eddy_adv_ml' : (('ocean_time','eta_rho', 'xi_rho'), eddy_adv_ml[:, 60:350, 60:1062]),
                            'SSTA' : (('ocean_time','eta_rho', 'xi_rho'), SSTA[:, 60:350, 60:1062]),
                            'x_rho' : (('eta_rho', 'xi_rho'), x_rho[60:350, 60:1062]),
                            'y_rho' : (('eta_rho', 'xi_rho'), y_rho[60:350, 60:1062])},
                coords={ 'eta_rho':   y_rho[ 60:350, 0], 'xi_rho':    x_rho[0, 60:1062]})
MS_ds

<xarray.Dataset>
Dimensions:      (eta_rho: 290, ocean_time: 12, xi_rho: 1002)
Coordinates:
  * eta_rho      (eta_rho) int64 200000 202500 205000 ... 917500 920000 922500
  * xi_rho       (xi_rho) int64 150000 152500 155000 ... 2647500 2650000 2652500
Dimensions without coordinates: ocean_time
Data variables:
    eddy_adv_ml  (ocean_time, eta_rho, xi_rho) float64 -1.24e+04 ... 889.7
    SSTA         (ocean_time, eta_rho, xi_rho) float64 0.07022 ... 0.02797
    x_rho        (eta_rho, xi_rho) int64 150000 152500 ... 2650000 2652500
    y_rho        (eta_rho, xi_rho) int64 200000 200000 200000 ... 922500 922500

## coefficients for binned statistics

In [7]:
population_cutoff = 5000
eddy_adv_bins = np.arange(-1700, 1700, 100)

# Eddy adv and SSTA

In [8]:
meso_grouped_mean = MS_ds.groupby_bins('eddy_adv_ml', eddy_adv_bins, squeeze=False).mean()
meso_grouped_std = MS_ds.groupby_bins('eddy_adv_ml', eddy_adv_bins, squeeze=False).std()
# calculating bin population
meso_groups = MS_ds.groupby_bins('eddy_adv_ml', eddy_adv_bins, squeeze=False).groups
count = 0
meso_bin_popu = np.zeros([len(meso_groups.keys()), 2])
for key in meso_groups.keys():
    meso_bin_popu[count] = key.left, len(meso_groups[key])
    count += 1
# finding the bins 
meso_valid = np.where(meso_bin_popu[meso_bin_popu[:, 0].argsort(), 1] > population_cutoff)

In [9]:
meso_grouped_mean

<xarray.Dataset>
Dimensions:           (eddy_adv_ml_bins: 33)
Coordinates:
  * eddy_adv_ml_bins  (eddy_adv_ml_bins) object (-1700, -1600] ... (1500, 1600]
Data variables:
    eddy_adv_ml       (eddy_adv_ml_bins) float64 -1.647e+03 ... 1.547e+03
    SSTA              (eddy_adv_ml_bins) float64 -0.02885 -0.02777 ... 0.1883
    x_rho             (eddy_adv_ml_bins) float64 1.277e+06 ... 1.443e+06
    y_rho             (eddy_adv_ml_bins) float64 2.173e+05 ... 2.217e+05

In [10]:
eddy_adv_ml_data, MS_SSTA_mean_data, MS_SSTA_std_data = meso_grouped_mean.eddy_adv_ml[meso_valid].data, meso_grouped_mean.SSTA[meso_valid].data, meso_grouped_std.SSTA[meso_valid].data
MS_bin_popu_data = meso_bin_popu[meso_bin_popu[:, 0].argsort(), 1][meso_valid]
#mean_adv_ml_data, LS_SSTA_mean_data, LS_SSTA_std_data, LS_bin_popu_data

In [11]:
eddy_stats_ds = xr.Dataset(data_vars={'eddy_adv_ml_bin_left' : (('temp_adv'), eddy_adv_ml_data),
                           'SSTA_mean' : (('temp_adv'), MS_SSTA_mean_data),
                           'SSTA_std' : (('temp_adv'), MS_SSTA_std_data),
                           'eddy_adv_ml_bin_population' : (('temp_adv'), MS_bin_popu_data)},
                coords={ 'eddy_adv': np.arange(len(MS_bin_popu_data))})
eddy_stats_ds

<xarray.Dataset>
Dimensions:                     (eddy_adv: 22, temp_adv: 22)
Coordinates:
  * eddy_adv                    (eddy_adv) int64 0 1 2 3 4 5 ... 17 18 19 20 21
Dimensions without coordinates: temp_adv
Data variables:
    eddy_adv_ml_bin_left        (temp_adv) float64 -1.148e+03 ... 945.3
    SSTA_mean                   (temp_adv) float64 -0.01471 -0.01108 ... 0.08905
    SSTA_std                    (temp_adv) float64 0.09743 0.1044 ... 0.1217
    eddy_adv_ml_bin_population  (temp_adv) float64 5.75e+03 ... 6.294e+03

In [12]:
eddy_stats_ds.to_netcdf('./' + case_name + '_SSTA_eddy_adv_300km_filter_binned_statistics_2016.nc')